In [113]:
from sympy import *

In [114]:
from icecream import ic
import pandas as pd

In [115]:
x = symbols('x')
y = Function('y')

In [116]:
yPrim = y(x).diff(x)
yPrim

Derivative(y(x), x)

In [117]:
def calculateDifferentialEquationWithDsolve(equation, initialCondition, pointValues = ()):
    solution = dsolve(equation, y(x), ics=initialCondition)
    solution = solution.simplify()
    values = [round(solution.rhs.subs(x, p).evalf(10), 5) for p in pointValues]
    return solution, values

In [118]:
points = [0.0, 0.2, 0.4, 0.6, 0.8, 1.0]

In [119]:
a1Equation = Eq(yPrim, 2 * y(x) + exp(x) - x)
a1InitialCondition = {y(0): 1 / 4}
a1Solution, a1Values = calculateDifferentialEquationWithDsolve(a1Equation, a1InitialCondition, points)
a1Solution, a1Values

(Eq(y(x), x/2 + exp(2*x) - exp(x) + 1/4),
 [0.25000, 0.62042, 1.18372, 2.04800, 3.37749, 5.42077])

In [120]:
b1Equation = Eq(y(x).diff(x) - (x + 1)*y(x), x - 1)
b1InitialCondition = {y(0): 0}
b1Solution, b1Values = calculateDifferentialEquationWithDsolve(b1Equation, b1InitialCondition, points)
b1Solution, b1Values

(Eq(y(x), (1 + sqrt(2)*sqrt(pi)*exp(1/2)*erf(sqrt(2)/2))*exp(x*(x + 2)/2) - sqrt(2)*sqrt(pi)*exp(x**2/2 + x + 1/2)*erf(sqrt(2)*(x + 1)/2) - 1),
 [0.0, -0.20283, -0.42446, -0.69114, -1.04407, -1.55268])

In [121]:
def createTable(tableName, columnHeaders, rowHeaders, tableValues):
    columns = [f"x{i} = {v}" for i, v in enumerate(columnHeaders)]
    table = pd.DataFrame(tableValues, columns=columns, index=rowHeaders)
    table.columns.name = tableName
    return table


In [122]:
aTable = createTable("(a)", points, ["RD", "RA", "RN"], [a1Values, [], []])
aTable

(a),x0 = 0.0,x1 = 0.2,x2 = 0.4,x3 = 0.6,x4 = 0.8,x5 = 1.0
RD,0.25000,0.62042,1.18372,2.04800,3.37749,5.42077
RA,None,None,None,None,None,None
RN,None,None,None,None,None,None


In [123]:
bTable = createTable("(b)", points, ["RD", "RA", "RN"], [b1Values, [], []])
bTable

(b),x0 = 0.0,x1 = 0.2,x2 = 0.4,x3 = 0.6,x4 = 0.8,x5 = 1.0
RD,0.0,-0.20283,-0.42446,-0.69114,-1.04407,-1.55268
RA,None,None,None,None,None,None
RN,None,None,None,None,None,None
